In [3]:
import requests
from bs4 import BeautifulSoup
import time

In [5]:
url = "https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
valid_links = ["/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"]
for i in range(100):
    valid_links = list(set(valid_links))
    baselink = "https://pt.wikipedia.org"
    url = f"{baselink}{valid_links[i]}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    todos_links = soup.find(id="bodyContent").find_all("a")
    for link in todos_links:
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if link["href"].find(":") == -1:
                valid_links.append(link["href"])
                print(valid_links[i])
    time.sleep(1)


In [19]:
import requests
from bs4 import BeautifulSoup

def coletar_links_wikipedia(inicio_url, limite=10):
    fila = [inicio_url]
    visitados = set()  
    links_coletados = [] 
    
    while fila and len(links_coletados) < limite:
        url_atual = fila.pop()
        
        if url_atual in visitados:
            continue 
        
        visitados.add(url_atual)
        links_coletados.append(url_atual)

        # Extrai os links da página
        response = requests.get(url_atual)
        soup = BeautifulSoup(response.text, "html.parser")
        for link in soup.find_all("a", href=True):
            verbete = link["href"]
            if verbete.startswith("/wiki/") and link["href"].startswith("/wiki/") and link["href"].find(":") == -1: # Filtra verbetes válidos
                novo_link = "https://pt.wikipedia.org" + verbete
                if novo_link not in visitados:
                    fila.append(novo_link)  
        
    return links_coletados


url_inicial = "https://pt.wikipedia.org/wiki/Wikipédia:Página_principal"
links = coletar_links_wikipedia(url_inicial)

print(links[:10])


['https://pt.wikipedia.org/wiki/Wikipédia:Página_principal', 'https://pt.wikipedia.org/wiki/Licen%C3%A7a_livre', 'https://pt.wikipedia.org/wiki/Encyclop%C3%A6dia_Britannica', 'https://pt.wikipedia.org/wiki/Biblioteca_Nacional_de_Israel', 'https://pt.wikipedia.org/wiki/Sistema_Universit%C3%A1rio_de_Documenta%C3%A7%C3%A3o', 'https://pt.wikipedia.org/wiki/Online_Computer_Library_Center', 'https://pt.wikipedia.org/wiki/Biblioteca_Nacional_da_Pol%C3%B3nia', 'https://pt.wikipedia.org/wiki/Registo_da_Mem%C3%B3ria_do_Mundo', 'https://pt.wikipedia.org/wiki/Lista_do_Patrim%C3%B4nio_Mundial_em_Perigo', 'https://pt.wikipedia.org/wiki/Organiza%C3%A7%C3%A3o_das_Na%C3%A7%C3%B5es_Unidas_para_a_Educa%C3%A7%C3%A3o,_a_Ci%C3%AAncia_e_a_Cultura']
